In [0]:
import glob
import pandas as pd
from google.colab import files
from google.colab import drive
import numpy as np
from tqdm import tqdm
import re
import math
import warnings
warnings.filterwarnings('ignore')

#Load in data

In [3]:
drive.mount('/content/drive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/Knab/Data/CleanData/"

ALS_inputs			     clean_page_only_articles.csv
clean_article_data.csv		     data_recommendations.csv
clean_article_data.gsheet	     lookup_urls.csv
clean_event_data.csv		     Popularity_score.csv
clean_page_data.csv		     ReadYN500.csv
clean_page_data_recommendations.csv  seen_recommendation.csv
clean_page_data_recom_timedelta.csv  time_delta.csv


In [0]:
input_recommendation = pd.read_csv("/content/drive/My Drive/Knab/Data/CleanData/data_recommendations.csv")
input_clean_page_data = pd.read_csv("/content/drive/My Drive/Knab/Data/CleanData/clean_page_data.csv")
input_seen_recommendation = pd.read_csv("/content/drive/My Drive/Knab/Data/CleanData/seen_recommendation.csv") #event data

In [0]:
df_seen_recommendation = input_seen_recommendation.drop(['hitnumber'], axis = 1)
df_seen_recommendation['seen_recommendations'] = 1

In [0]:
#The old recommendations got converted to strings when loading the data
def string_to_array(string):
  string = string.replace('[', '').replace(']', '').replace("'", "").replace(' ', '').split('\r\n')
  
  return string

In [0]:
input_recommendation['old_recommendations'] = input_recommendation['old_recommendations'].apply(string_to_array)

In [0]:
columns_rec = pd.DataFrame(input_recommendation.old_recommendations.tolist(), columns=['rec_1', 'rec_2', 'rec_3'])
df_recommended_articles =  pd.concat([input_recommendation,columns_rec], axis=1).drop(['old_recommendations'], axis = 1)

In [0]:
df_data = pd.merge(left = input_clean_page_data, right = df_recommended_articles[['URL', 'clientid_hashed', 'visitid', 'hitnumber', 'ArticleYN', 'rec_1', 'rec_2', 'rec_3']], left_on = ['URL', 'clientid_hashed', 'visitid', 'hitnumber'], right_on= ['URL', 'clientid_hashed', 'visitid', 'hitnumber'], how='left')
df_data = pd.merge(left = df_data, right = df_seen_recommendation, left_on = ['URL', 'clientid_hashed', 'visitid'], right_on = ['URL', 'clientid_hashed', 'visitid'], how='left').fillna(0)
df_data = df_data.drop_duplicates(['URL', 'clientid_hashed', 'visitid', 'hitnumber', 'visitstarttime'])

In [0]:
df_data_shifted = df_data[['clientid_hashed', 'URL']].rename(columns={'clientid_hashed': 'clientid_hashed_right', 'URL': 'URL_right'})

In [0]:
df_data_shift_left = df_data.drop(df_data.tail(1).index).reset_index(drop=True)
df_data_shift_right = df_data_shifted.drop(df_data_shifted.head(1).index).reset_index(drop=True)
df_click_check = pd.concat([df_data_shift_left,df_data_shift_right], axis=1)
df_click_check['same_client'] = np.where(((df_click_check['clientid_hashed'] == df_click_check['clientid_hashed_right'])),1,0)
df_click_check = df_click_check[df_click_check['same_client'] == 1]
df_click_check = df_click_check[df_click_check['BiebYN'] == 1]
df_click_check = df_click_check[df_click_check['ArticleYN'] == 1]
df_click_check = df_click_check[df_click_check['seen_recommendations'] == 1]

In [0]:
df_click_check['clicked_recommendation'] = 0
df_click_check = df_click_check.drop(['clientid_hashed_right', 'channelgrouping', 'browser', 'devicecategory'], axis = 1)
df_click_check.loc[(df_click_check['URL_right'] == df_click_check['rec_1']), 'clicked_recommendation'] = 'rec_1'
df_click_check.loc[(df_click_check['URL_right'] == df_click_check['rec_2']), 'clicked_recommendation'] = 'rec_2'
df_click_check.loc[(df_click_check['URL_right'] == df_click_check['rec_3']), 'clicked_recommendation'] = 'rec_3'
df_click_check = df_click_check.drop_duplicates(['URL', 'clientid_hashed', 'visitid', 'hitnumber', 'visitstarttime'],keep='last')

In [0]:
df_data = pd.merge(left = df_data, right = df_click_check[['URL', 'clientid_hashed', 'visitid', 'hitnumber', 'clicked_recommendation']], left_on = ['URL', 'clientid_hashed', 'visitid', 'hitnumber'], right_on= ['URL', 'clientid_hashed', 'visitid', 'hitnumber'], how='left')
df_data = df_data.drop_duplicates(['URL', 'clientid_hashed', 'visitid', 'hitnumber', 'visitstarttime']).fillna(0)

In [0]:
df_data['clicked_url'] = 0
df_data.loc[df_data['clicked_recommendation'] == 'rec_1', 'clicked_url'] = df_data['rec_1']
df_data.loc[df_data['clicked_recommendation'] == 'rec_2', 'clicked_url'] = df_data['rec_2']
df_data.loc[df_data['clicked_recommendation'] == 'rec_3', 'clicked_url'] = df_data['rec_3']

In [0]:
count_list_recommended = df_data[(df_data['clicked_recommendation'] != 0) & (df_data['URL'] != df_data['clicked_url'])]['clicked_url'].value_counts()

In [0]:
df_data.to_csv("/content/drive/My Drive/Knab/Data/CleanData/clean_page_data_recommendations.csv", index = False)

In [0]:
df_clean_page_data_recom = pd.read_csv("/content/drive/My Drive/Knab/Data/CleanData/clean_page_data_recommendations.csv")

In [0]:
df_timedelta = pd.read_csv("/content/drive/My Drive/Knab/Data/CleanData/time_delta.csv")

In [0]:
df_clean_page_data_recom_timedelta = pd.merge(left = df_clean_page_data_recom, right = df_timedelta[['URL', 'clientid_hashed', 'visitid', 'hitnumber','delta_time']], left_on = ['URL', 'clientid_hashed', 'visitid', 'hitnumber'], right_on = ['URL', 'clientid_hashed', 'visitid', 'hitnumber'], how = 'left')

In [0]:
df_clean_page_data_recom_timedelta.to_csv("/content/drive/My Drive/Knab/Data/CleanData/clean_page_data_recom_timedelta.csv", index = False)

#Checking 'biased' articles

In [0]:
df_clean_page_data_recom_timedelta = pd.read_csv("/content/drive/My Drive/Knab/Data/CleanData/clean_page_data_recom_timedelta.csv")

In [0]:
df_work = df_clean_page_data_recom_timedelta

In [0]:
df_work = df_work[df_work['ArticleYN'] == 1.0] # only take data points which are article clicks

In [0]:
df_click_recom = df_work[df_work['clicked_url'] != "0"]['clicked_url'].value_counts().reset_index().rename(columns = {'index' : 'URL', 'clicked_url' : 'count_rec'})

In [0]:
df_click_bieb = df_work['URL'].value_counts().reset_index().rename(columns = {'index' : 'URL', 'URL' : 'count'})

In [0]:
df_rec1 = df_work['rec_1'].value_counts().reset_index().rename(columns = {'index' : 'URL', 'URL' : 'count_rec1'})
df_rec2 = df_work['rec_2'].value_counts().reset_index().rename(columns = {'index' : 'URL', 'URL' : 'count_rec2'})
df_rec3 = df_work['rec_3'].value_counts().reset_index().rename(columns = {'index' : 'URL', 'URL' : 'count_rec3'})

In [0]:
#Merge all Article clicks (df_click_bieb), no. times recommended (df_rec1,2,3) and no. clicked on recommendation (df_click_recom)
df_bias_counts = df_click_bieb.merge(df_click_recom, how = "left").merge(df_rec1, how = "left").merge(df_rec2, how = "left").merge(df_rec3, how = "left").fillna(0)
df_bias_counts['number_rec'] = df_bias_counts['rec_1'] + df_bias_counts['rec_2'] + df_bias_counts['rec_3']
df_bias_counts = df_bias_counts.drop(['rec_1', 'rec_2', 'rec_3'], axis = 1)

In [0]:
#Calculate probability of being clicked given being recommended
df_bias_counts['prob_click_given_rec'] = np.where((df_bias_counts['number_rec'] > 0), df_bias_counts['count_rec']/df_bias_counts['number_rec'], 0)

In [0]:
#Calculate probability of being clicked given not being recommended
df_bias_counts['prob_click_given_not_rec'] = (df_bias_counts['count'] - df_bias_counts['count_rec'])/ (df_bias_counts['count'].values.sum() - df_bias_counts['number_rec'])

In [0]:
#Calculate bias = difference in probabilities
df_bias_counts['article_bias'] = 100*(df_bias_counts['prob_click_given_rec'] - df_bias_counts['prob_click_given_not_rec'])

In [0]:
#Average bias among URLs that are recommended ( > 0)
avg_bias = np.mean(df_bias_counts[df_bias_counts['number_rec'] > 0]['article_bias'].values)

In [0]:
#Weighted average bias among URLs that are recommended, weighted on fraction of clicks on total clicks of recommended articles
weighted_avg_bias = np.average(df_bias_counts[df_bias_counts['number_rec'] > 0]['article_bias'].values,weights = df_bias_counts[df_bias_counts['number_rec'] > 0]['count'].values/df_bias_counts[df_bias_counts['number_rec'] > 0]['count'].values.sum())

In [0]:
# t_score of weighted average bias
z_score = weighted_avg_bias/np.std(df_bias_counts[df_bias_counts['number_rec'] > 0]['article_bias'].values)

In [0]:
print(f"Average bias: {avg_bias}")
print(f"Weighted average bias: {weighted_avg_bias}")
print(f"Z score: {z_score}")

Average bias: 0.3749366713099083
Weighted average bias: 0.17757059548574392
Z score: 0.26202138474699355


In [0]:
df_bias_counts

,URL,count,count_rec,number_rec,prob_click_given_rec,prob_click_given_not_rec,article_bias
0,bieb.knab.nl/sparen-investeren/belastingvrij-s...,48514,0.0,0.0,0.000000,4.230635e-02,-4.230635
1,bieb.knab.nl/hypotheken/overwaarde-huis-opneme...,20429,0.0,0.0,0.000000,1.781499e-02,-1.781499
2,bieb.knab.nl/inkomsten-uitgaven/prinsjesdag-20...,19521,274.0,13524.0,0.020260,1.698454e-02,0.327574
3,bieb.knab.nl/hypotheken/werkgeversverklaring-e...,18722,0.0,0.0,0.000000,1.632641e-02,-1.632641
4,bieb.knab.nl/hypotheken/bieden-op-een-huis-voo...,18606,954.0,37655.0,0.025335,1.591595e-02,0.941933
5,bieb.knab.nl/hypotheken/prinsjesdag-2018-dit-v...,15602,88.0,17775.0,0.004951,1.374190e-02,-0.879113
6,bieb.knab.nl/hypotheken/intrekken-koophuis-sam...,13000,310.0,15910.0,0.019485,1.122194e-02,0.826267
7,bieb.knab.nl/sparen-investeren/vermogensbelast...,12031,39.0,6064.0,0.006431,1.051315e-02,-0.408175
8,bieb.knab.nl/hypotheken/huis-kopen-om-te-verhu...,11269,35.0,5369.0,0.006519,9.842627e-03,-0.332372
9,bieb.knab.nl/hypotheken/huis-kopen-eigen-geld-...,11056,11.0,2963.0,0.003712,9.656679e-03,-0.594423
